## Accessing NOAA U.S. Tabular Climate Normals with the Planetary Computer STAC API

The [NOAA U.S. Tabular Climate Normals](https://www.ncei.noaa.gov/products/land-based-station/us-climate-normals) provide information about typical climate conditions for thousands of weather station locations across the United States. Normals act both as a ruler to compare current weather and as a predictor of conditions in the near future. The official normals are calculated for a uniform 30 year period, and consist of annual/seasonal, monthly, daily, and hourly averages and statistics of temperature, precipitation, and other climatological variables for each weather station.

This Collection contains tabular weather variable data at weather station locations in GeoParquet format. Data are provided for annual/seasonal, monthly, daily, and hourly frequencies for the following time periods:

- 30-year Conventional (1991–2020)
- 30-year Legacy (1981–2010)
- 15-year Supplemental (2006–2020)

Documentation for this dataset is available at the [Planetary Computer Data Catalog](https://planetarycomputer.microsoft.com/dataset/noaa-climate-normals-tabular).


### Data Access
This notebook works with or without an API key, but you will be given more permissive access to the data with an API key. The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) sets the environment variable "PC_SDK_SUBSCRIPTION_KEY" when your server is started. The API key may be manually set via the following code:

```python
pc.settings.set_subscription_key(<YOUR API Key>)
```

The datasets hosted by the Planetary Computer are available from [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/). We'll use [pystac-client](https://pystac-client.readthedocs.io/) to search the Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1/docs) for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a `modifier` so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See [Reading from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) and [Using tokens for data access](https://planetarycomputer.microsoft.com/docs/concepts/sas/) for more. 

In [1]:
import planetary_computer
import pystac_client

# Open the Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://planetarycomputer-staging.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

collection = catalog.get_collection("noaa-climate-normals-tabular")
collection

ID: noaa-climate-normals-tabular
Title: NOAA US Tabular Climate Normals
"Description: The [NOAA United States Climate Normals](https://www.ncei.noaa.gov/products/land-based-station/us-climate-normals) provide information about typical climate conditions for thousands of weather station locations across the United States. Normals act both as a ruler to compare current weather and as a predictor of conditions in the near future. The official normals are calculated for a uniform 30 year period, and consist of annual/seasonal, monthly, daily, and hourly averages and statistics of temperature, precipitation, and other climatological variables for each weather station. NOAA produces Climate Normals in accordance with the [World Meteorological Organization](https://public.wmo.int/en) (WMO), of which the United States is a member. The WMO requires each member nation to compute 30-year meteorological quantity averages at least every 30 years, and recommends an update each decade, in part to incorporate newer weather stations. The 1991–2020 U.S. Climate Normals are the latest in a series of decadal normals first produced in the 1950s. This Collection contains tabular weather variable data at weather station locations in GeoParquet format. Data are provided for annual/seasonal, monthly, daily, and hourly frequencies for the following time periods: - Conventional 30-year normals (1991–2020) - Legacy 30-year normals (1981–2010) - Supplemental 15-year normals (2006–2020)"
"Providers: NOAA National Centers for Environmental Information (producer, processor, licensor) Microsoft (processor, host)"
type: Collection
title: NOAA US Tabular Climate Normals
"assets: {'thumbnail': {'href': 'https://ai4edatasetspublicassets.blob.core.windows.net/assets/pc_thumbnails/noaa-climate-normals-tabular-thumb.png', 'type': 'image/png', 'roles': ['thumbnail'], 'title': 'NOAA U.S. Tabular Climate Normals Thumbnail'}, 'geoparquet-items': {'href': 'abfs://items/noaa-climate-normals-tabular.parquet', 'type': 'application/x-parquet', 'roles': ['stac-items'], 'title': 'GeoParquet STAC items', 'description': ""Snapshot of the collection's STAC items exported to GeoParquet format."", 'msft:partition_info': {'is_partitioned': False}, 'table:storage_options': {'account_name': 'pcstacitems', 'credential': 'st=2023-01-12T21%3A21%3A19Z&se=2023-01-20T21%3A21%3A20Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-01-13T21%3A21%3A18Z&ske=2023-01-20T21%3A21%3A18Z&sks=b&skv=2021-06-08&sig=vDoScOCntdyRyakGmuI48/0gHptx0fuaQUZ/rOXv2v0%3D'}}}"
"keywords: ['NOAA', 'Climate Normals', 'Weather', 'Surface Observations', 'Climatology', 'CONUS']"
"providers: [{'url': 'https://www.ncei.noaa.gov/', 'name': 'NOAA National Centers for Environmental Information', 'roles': ['producer', 'processor', 'licensor']}, {'url': 'https://planetarycomputer.microsoft.com', 'name': 'Microsoft', 'roles': ['processor', 'host']}]"
"summaries: {'noaa_climate_normals:period': ['1981-2010', '1991-2020', '2006-2020'], 'noaa_climate_normals:frequency': ['hourly', 'daily', 'monthly', 'annualseasonal']}"
"item_assets: {'geoparquet': {'type': 'application/x-parquet', 'roles': ['data'], 'title': 'Dataset root', 'table:storage_options': {'account_name': 'noaanormals'}}}"


In [2]:
# Get Items
items = list(collection.get_all_items())
items

[<Item id=2006_2020-monthly>,
 <Item id=2006_2020-hourly>,
 <Item id=2006_2020-daily>,
 <Item id=2006_2020-annualseasonal>,
 <Item id=1991_2020-monthly>,
 <Item id=1991_2020-hourly>,
 <Item id=1991_2020-daily>,
 <Item id=1991_2020-annualseasonal>,
 <Item id=1981_2010-monthly>,
 <Item id=1981_2010-hourly>,
 <Item id=1981_2010-daily>,
 <Item id=1981_2010-annualseasonal>]

### Available Assets & Metadata

Let's display the available assets and metadata for the NOAA U.S. Tabular Climate Normals product. 

In [3]:
import rich.table

# Assets
t_assets = rich.table.Table("Key", "Value")
for key, asset in items[0].assets.items():
    t_assets.add_row(key, asset.title)
t_assets

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Key        ┃ Value        ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ geoparquet │ Dataset root │
└────────────┴──────────────┘

In [4]:
# Metadata
t_metadata = rich.table.Table("Key", "Value")
for k, v in sorted(items[0].properties.items()):
    if k != "table:columns":
        t_metadata.add_row(k, str(v))
t_metadata

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Key                            ┃ Value                                                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ created                        │ 2023-01-13T17:12:43.569562Z                                                    │
│ datetime                       │ None                                                                           │
│ end_datetime                   │ 2020-12-31T23:59:59Z                                                           │
│ noaa_climate_normals:frequency │ monthly                                                                        │
│ noaa_climate_normals:period    │ 2006-2020                                                                      │
│ proj:epsg                      │ 4269                                                                           │
│ sci:publications               │ [{'doi': '10.1175/BAMS-D-11-00197.1', 'citation': "Arguez, A., I. Durre, S.    │
│                                │ Applequist, R. Vose, M. Squires, X. Yin, R. Heim, and T. Owen, 2012: NOAA's    │
│                                │ 1981-2010 climate normals: An overview. Bull. Amer. Meteor. Soc., 93,          │
│                                │ 1687-1697. DOI: 10.1175/BAMS-D-11-00197.1."}]                                  │
│ start_datetime                 │ 2006-01-01T00:00:00Z                                                           │
│ table:row_count                │ 161652                                                                         │
│ title                          │ Monthly Climate Normals for Period 2006-2020                                   │
└────────────────────────────────┴────────────────────────────────────────────────────────────────────────────────┘

### Loading the GeoParquet dataset
Now let's load a STAC item into a tabular format. 

In [5]:
import geopandas

df = geopandas.read_parquet(
    asset.href,
    storage_options=asset.extra_fields["table:storage_options"],
    columns=["STATION", "month", "MLY-PRCP-NORMAL", "geometry"],
)

### Displaying the tabular data

Let's display the NOAA U.S. Tabular Climate Normals for the Continental United States and Puerto Rico.

In [6]:
import contextily

ax = df.cx[-125:24, -65:50.5].plot(figsize=(24, 24), column="MLY-PRCP-NORMAL")
contextily.add_basemap(
    ax, crs=df.crs.to_string(), source=contextily.providers.OpenStreetMap.Mapnik
)

<Figure size 2400x2400 with 1 Axes>